In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
df=pd.read_csv(r'C:\Users\Hrushikesh\OneDrive\Desktop\ML Hackathon\student_resource 3\dataset\mapped_train.csv')

In [3]:
df

,Unnamed: 0,image_link,group_id,entity_value,Image,Image_Arrays,OCR_Text,entity_number,entity_name_item_volume,entity_name_item_weight,...,entity_unit_cup,entity_unit_gallon,entity_unit_gram,entity_unit_kilogram,entity_unit_milligram,entity_unit_millilitre,entity_unit_ounce,entity_unit_pound,entity_unit_volt,entity_unit_watt
0,0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,500.0 gram,<PIL.Image.Image image mode=RGB size=1600x1600...,[[[255 255 255]\n [255 255 255]\n [255 255 2...,100% NATUR\n\n,500.000,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,1.0 cup,<PIL.Image.Image image mode=RGB size=1200x1200...,[[[155 190 209]\n [153 188 207]\n [153 188 2...,GEPRAGTES\n\nDesigned in\n\nLIZENZIERTE UND GE...,1.000,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,0.709 gram,<PIL.Image.Image image mode=RGB size=1081x1081...,[[[255 255 255]\n [255 255 255]\n [255 255 2...,Serving Size: 1 Tablet (0.709 g) | Each servin...,0.709,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,0.709 gram,<PIL.Image.Image image mode=RGB size=1081x1081...,[[[255 255 255]\n [255 255 255]\n [255 255 2...,NaN,0.709,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,1400 milligram,<PIL.Image.Image image mode=RGB size=1500x1500...,[[[255 255 255]\n [255 255 255]\n [255 255 2...,HIGH STRENGTH\n\nPSYLLIUM\n\nHUSK\n\n,1400.000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,https://m.media-amazon.com/images/I/71XK5d3Oh9...,416664,49.0 watt,<PIL.Image.Image image mode=RGB size=2560x2560...,[[[255 255 255]\n [255 255 255]\n [255 255 2...,NaN,49.000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
96,96,https://m.media-amazon.com/images/I/61kyBEJYDe...,459516,500 milligram,<PIL.Image.Image image mode=RGB size=1500x1500...,[[[255 255 255]\n [255 255 255]\n [255 255 2...,NaN,500.000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
97,97,https://m.media-amazon.com/images/I/71uQmsTESv...,459516,500 milligram,<PIL.Image.Image image mode=RGB size=1500x1500...,[[[253 253 253]\n [254 254 254]\n [254 254 2...,GREEN COFFEE\n\nDiscover weliness\n\nS42\n6)\n...,500.000,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
98,98,https://m.media-amazon.com/images/I/71jG8BOi4W...,241608,16.0 gram,<PIL.Image.Image image mode=RGB size=1600x1600...,[[[227 228 230]\n [228 229 231]\n [228 229 2...,16G Thick High Grade 304 Stainless Steel\n\n— ...,16.000,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
import os
import numpy as np
import pandas as pd
from PIL import Image, UnidentifiedImageError, ImageFile

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import re

# Set up pytesseract



# Preprocess and resize image
def process_image(image, size=(244, 244)):
    try:
        # Resize the image
        image_resized = image.resize(size)
        # Convert to numpy array
        image_array = np.array(image_resized)
        return image_array
    except Exception as e:
        print(f"Failed to process image: {e}")
        return None


# OCR function to extract text from the image
def extract_text_from_image(image):
    try:
        text = pytesseract.image_to_string(image)
        return text
    except Exception as e:
        print(f"Error extracting text from image: {e}")
        return ""


# Function to load images and apply OCR
def load_images_with_ocr(df, image_dir, max_images, batch_size=100):
    total_images = len(df)
    batches = (total_images + batch_size - 1) // batch_size  # Calculate number of batches
    images = []
    image_arrays = []
    all_images = []
    ocr_texts = []  # To store OCR results
    df = df.head(max_images)

    for batch_index in range(batches):
        batch_df = df.iloc[batch_index * batch_size:(batch_index + 1) * batch_size]
        successfully_loaded = 0
        skipped_files = 0
        total_mapped = 0

        for index, row in batch_df.iterrows():
            total_mapped += 1
            # Extract the filename from the image link
            image_filename = row['image_link'].split('/')[-1]
            image_path = os.path.join(image_dir, image_filename)
            image_path = image_path.replace("\\", "/")

            # Load the image if it exists, else append None
            if os.path.exists(image_path):
                try:
                    with Image.open(image_path) as img:
                        # Resize image
                        image_array = process_image(img)
                        image_arrays.append(image_array)

                        # Apply OCR to extract text
                        ocr_text = extract_text_from_image(img)
                        ocr_texts.append(ocr_text)

                        images.append(img.copy())  # Copy the image to avoid keeping the file open
                        successfully_loaded += 1
                except (UnidentifiedImageError, OSError) as e:
                    print(f"Warning: Error opening image file {image_path}: {e}")
                    images.append(None)  # Append None if the image is invalid or corrupted
                    ocr_texts.append("")  # Append empty string for OCR result if error
                    skipped_files += 1
            else:

                print(f"Warning: Image file {image_path} does not exist. Skipping...")
                images.append(None)
                ocr_texts.append("")  # No OCR text for missing images
                skipped_files += 1
        all_images.extend(images)
        print(f"Total images mapped: {total_mapped}")
        print(f"Successfully loaded images: {successfully_loaded}")
        print(f"Skipped images: {skipped_files}")

    return images, image_arrays, ocr_texts


# Load training data
train_df = pd.read_csv(r'C:\Users\Hrushikesh\OneDrive\Desktop\ML Hackathon\student_resource 3\dataset\mapped_train.csv')

train_image_dir = 'downloaded_images_train'
MAX_IMAGES = 100

# Load training images and OCR text
train_images, train_arrays, ocr_texts = load_images_with_ocr(train_df, train_image_dir, MAX_IMAGES, 100)

# Add OCR text to the dataset
mapped_train_dataset = train_df.head(MAX_IMAGES)
mapped_train_dataset['Image'] = train_images
mapped_train_dataset['Image_Arrays'] = train_arrays
mapped_train_dataset['OCR_Text'] = ocr_texts

# Step 1: Split 'entity_value' into numeric value and unit
mapped_train_dataset[['entity_number', 'entity_unit']] = mapped_train_dataset['entity_value'].str.extract(r'([0-9.]+)\s*(\w+)')

# Step 2: Convert the numeric part to float
mapped_train_dataset['entity_number'] = mapped_train_dataset['entity_number'].astype(float)

# Display the resulting dataframe
print(mapped_train_dataset[['OCR_Text','entity_value', 'entity_number', 'entity_unit']])

# # Step 2: Convert the numeric part to float
# mapped_train_dataset['entity_number'] = pd.to_numeric(mapped_train_dataset['entity_number'], errors='coerce')

# # Display the resulting dataframe with OCR text, numeric value, and units
# print(mapped_train_dataset[['OCR_Text', 'entity_number', 'entity_unit']])

# # Drop the 'entity_value' column (if exists) and handle the rest of the dataset
# mapped_train_dataset = mapped_train_dataset.drop(columns='entity_value', errors='ignore')

# One-hot encode the 'entity_name' and 'entity_unit' columns
encoder = OneHotEncoder()

# Encode entity_name
categorical_column = mapped_train_dataset[['entity_name']]
column_encoded = encoder.fit_transform(categorical_column)
column_encoded_df = pd.DataFrame(column_encoded.toarray(), columns=encoder.get_feature_names_out(['entity_name']))
mapped_train_dataset = pd.concat([mapped_train_dataset.drop(columns=['entity_name']), column_encoded_df], axis=1)

# Encode entity_unit
categorical_column = mapped_train_dataset[['entity_unit']]
column_encoded = encoder.fit_transform(categorical_column)
column_encoded_df = pd.DataFrame(column_encoded.toarray(), columns=encoder.get_feature_names_out(['entity_unit']))
mapped_train_dataset = pd.concat([mapped_train_dataset.drop(columns=['entity_unit']), column_encoded_df], axis=1)


# Convert train_arrays (list) to a NumPy array
train_arrays = np.array(train_arrays)
# Flatten the image arrays
train_image_arrays_flat = train_arrays.reshape(train_arrays.shape[0], -1)

col_to_drop=['image_link','group_id','entity_value','Image','OCR_Text']

print(mapped_train_dataset.columns.tolist())


# Drop the columns from the DataFrame
mapped_train_dataset_dropped = mapped_train_dataset.drop(columns=col_to_drop, errors='ignore')

# Print the columns to ensure correct columns are dropped
print("Columns after dropping:")
print(mapped_train_dataset_dropped.columns.tolist())

# Combine image arrays and encoded categorical data
try:
    train_features = np.hstack((train_image_arrays_flat, mapped_train_dataset_dropped.to_numpy()))
    print("Shape of train_features:", train_features.shape)
except Exception as e:
    print(f"Error combining features: {e}")

# Extract target variable
target_number = mapped_train_dataset_dropped['entity_number']
target_unit = mapped_train_dataset_dropped.filter(regex='^entity_unit_')

# Ensure all data is numeric
mapped_train_dataset_dropped = mapped_train_dataset_dropped.apply(pd.to_numeric, errors='coerce')

# Convert train_arrays (list) to a NumPy array
train_arrays = np.array(train_arrays)

# Flatten the image arrays
train_image_arrays_flat = train_arrays.reshape(train_arrays.shape[0], -1)

# Drop the columns from the DataFrame
mapped_train_dataset_dropped = mapped_train_dataset_dropped.drop(columns=col_to_drop, errors='ignore')

# Combine image arrays and encoded categorical data
try:
    train_features = np.hstack((train_image_arrays_flat, mapped_train_dataset_dropped.to_numpy()))
    print("Shape of train_features:", train_features.shape)
except Exception as e:
    print(f"Error combining features: {e}")

# Check for NaNs and Infs
print("NaNs in train_features:", np.any(np.isnan(train_features)))
print("Infs in train_features:", np.any(np.isinf(train_features)))

# Replace NaNs with a specific value (e.g., 0)
train_features = np.nan_to_num(train_features)

# Proceed with train/test split and model fitting
X_train, X_valid, y_train_number, y_valid_number = train_test_split(train_features, target_number, test_size=0.2, random_state=42)
X_train, X_valid, y_train_unit, y_valid_unit = train_test_split(train_features, target_unit, test_size=0.2, random_state=42)

# Train and evaluate models
regressor = RandomForestRegressor(n_estimators=100, random_state=42)
regressor.fit(X_train, y_train_number)
y_pred_number = regressor.predict(X_valid)
print("Mean Squared Error for entity_number:", mean_squared_error(y_valid_number, y_pred_number))

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train_unit)
y_pred_unit = classifier.predict(X_valid)
print("Accuracy for entity_unit:", accuracy_score(y_valid_unit, y_pred_unit))

Total images mapped: 100
Successfully loaded images: 0
Skipped images: 100


ValueError: Length of values (0) does not match length of index (100)